José Luis García Nava
DEPFIE-SCOM
Cloud-based Implementation of Distributed Machine Learning Algorithms for Time Series Forecasting
Parquet Archive as Time Series Database Management System
Updated from MIRD-related research to Spark 3

In [108]:
SOURCE_PARQUET_PATH = '/home/developer/On_Premises/Data_Lake/complete_20160101000000_to_20180809114000'

In [109]:
RESOLUTION = 'hourly'

In [110]:
devices = ['CPE04015']

In [111]:
for device in devices:
    path = '{}/{}/{}.parquet'.format(SOURCE_PARQUET_PATH, RESOLUTION, devices[0])

In [112]:
# the DataFrame contains hourly data for multiple dates
raw_df = spark.read.parquet(path)

In [113]:
# change timestamp from string to datetime
# from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import to_timestamp

In [114]:
# a nice trick to get a string with remaining columns, for copy and paste
"', '".join(raw_df.columns[1:])

"Van', 'Vbn', 'Vcn', 'Vav', 'ia', 'ib', 'ic', 'iav', 'kw', 'kvar', 'kwan', 'kwbn', 'kwcn', 'kvaran', 'kvarbn', 'kvarcn', 'f', 'fp', 'thdvan', 'thdvbn', 'thdvcn', 'thdia', 'thdib', 'thdic', 'desbV', 'desbI', 'kwhE', 'kwhR', 'kvarhDel', 'kvarhrec', 'kvarhq3', 'kvarhq4"

In [115]:
# multidate_df is a Spark DataFrame with timestamp as datetime
# use a Spark DataFrame operation instead of a Spark SQL query
multidate_df = raw_df.select(to_timestamp(raw_df.timestamp, 'yyyy-MM-dd HH:mm:ss').alias('timestamp'),
                             'Van', 'Vbn', 'Vcn', 'Vav', \
                             'ia', 'ib', 'ic', 'iav', \
                             'kw', 'kvar', 'kwan', 'kwbn', 'kwcn', 'kvaran', 'kvarbn', 'kvarcn', \
                             'f', 'fp', 'thdvan', 'thdvbn', 'thdvcn', 'thdia', 'thdib', 'thdic', \
                             'desbV', 'desbI', \
                             'kwhE', 'kwhR', 'kvarhDel', 'kvarhrec', 'kvarhq3', 'kvarhq4')

In [116]:
# now timestamp column in Spark DataFrame is really a timestamp
multidate_df.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- Van: double (nullable = true)
 |-- Vbn: double (nullable = true)
 |-- Vcn: double (nullable = true)
 |-- Vav: double (nullable = true)
 |-- ia: double (nullable = true)
 |-- ib: double (nullable = true)
 |-- ic: double (nullable = true)
 |-- iav: double (nullable = true)
 |-- kw: double (nullable = true)
 |-- kvar: double (nullable = true)
 |-- kwan: double (nullable = true)
 |-- kwbn: double (nullable = true)
 |-- kwcn: double (nullable = true)
 |-- kvaran: double (nullable = true)
 |-- kvarbn: double (nullable = true)
 |-- kvarcn: double (nullable = true)
 |-- f: double (nullable = true)
 |-- fp: double (nullable = true)
 |-- thdvan: double (nullable = true)
 |-- thdvbn: double (nullable = true)
 |-- thdvcn: double (nullable = true)
 |-- thdia: double (nullable = true)
 |-- thdib: double (nullable = true)
 |-- thdic: double (nullable = true)
 |-- desbV: double (nullable = true)
 |-- desbI: double (nullable = true)
 |-- kwhE: doubl

In [119]:
# create the temporary view on the Spark DataFrame

In [120]:
multidate_df.createOrReplaceTempView('multidate_df_view')

In [121]:
# how many hour-based lectures in the DataFrame
spark.sql('select timestamp, kw \
           from multidate_df_view \
           order by timestamp').count()

22832

In [122]:
# a DataFrame with all dates in the hour-based DataFrame
date_df = spark.sql('select substring(timestamp, 1, 10) as date \
                     from multidate_df_view \
                     group by substring(timestamp, 1, 10) \
                     order by substring(timestamp, 1, 10)')

In [123]:
# all dates to an ordered list
dates_list = date_df.rdd.map(lambda row : row.date).collect()
dates_list.sort()

In [124]:
date = dates_list[0]

In [125]:
date

'2016-01-01'

In [143]:
# change the above query string to persist timestamp as string
query_string = 'select string(timestamp) as timestamp, \
                       Van, \
                       Vbn, \
                       Vcn, \
                       Vav, \
                       ia, \
                       ib, \
                       ic, \
                       iav, \
                       kw, \
                       kvar, \
                       kwan, \
                       kwbn, \
                       kwcn, \
                       kvaran, \
                       kvarbn, \
                       kvarcn, \
                       f, \
                       fp, \
                       thdvan, \
                       thdvbn, \
                       thdvcn, \
                       thdia, \
                       thdib, \
                       thdic, \
                       desbV, \
                       desbI, \
                       kwhE, \
                       kwhR, \
                       kvarhDel, \
                       kvarhrec, \
                       kvarhq3, \
                       kvarhq4 \
                from multidate_df_view \
                where substring(timestamp, 1, 10) = "{}" \
                order by substring(timestamp, 1, 10)'.format(date)

In [144]:
query_string

'select string(timestamp) as timestamp,                        Van,                        Vbn,                        Vcn,                        Vav,                        ia,                        ib,                        ic,                        iav,                        kw,                        kvar,                        kwan,                        kwbn,                        kwcn,                        kvaran,                        kvarbn,                        kvarcn,                        f,                        fp,                        thdvan,                        thdvbn,                        thdvcn,                        thdia,                        thdib,                        thdic,                        desbV,                        desbI,                        kwhE,                        kwhR,                        kvarhDel,                        kvarhrec,                        kvarhq3,                        kvarhq4                 from

In [145]:
# get lectures of a single date from the original DataFrame
buffer_df = spark.sql(query_string)

In [148]:
# build a path to the directory where date-based parquet archives reside

In [149]:
DEST_PARQUET_PATH = '/home/developer/On_Premises/MIRD_ROOT/data/raw'

In [150]:
archive_string = '{}/{}/{}.parquet/{}'.format(DEST_PARQUET_PATH, RESOLUTION, device, date)

In [151]:
archive_string

'/home/developer/On_Premises/MIRD_ROOT/data/raw/hourly/CPE04015.parquet/2016-01-01'

In [152]:
# now persist buffer_df, with a given max number of partitions
buffer_df.coalesce(2).write.parquet(archive_string)

In [153]:
# read the brand new DataFrame to verify it

In [154]:
test_df = spark.read.parquet(archive_string)

In [155]:
test_df.printSchema()

root
 |-- timestamp: string (nullable = true)
 |-- Van: double (nullable = true)
 |-- Vbn: double (nullable = true)
 |-- Vcn: double (nullable = true)
 |-- Vav: double (nullable = true)
 |-- ia: double (nullable = true)
 |-- ib: double (nullable = true)
 |-- ic: double (nullable = true)
 |-- iav: double (nullable = true)
 |-- kw: double (nullable = true)
 |-- kvar: double (nullable = true)
 |-- kwan: double (nullable = true)
 |-- kwbn: double (nullable = true)
 |-- kwcn: double (nullable = true)
 |-- kvaran: double (nullable = true)
 |-- kvarbn: double (nullable = true)
 |-- kvarcn: double (nullable = true)
 |-- f: double (nullable = true)
 |-- fp: double (nullable = true)
 |-- thdvan: double (nullable = true)
 |-- thdvbn: double (nullable = true)
 |-- thdvcn: double (nullable = true)
 |-- thdia: double (nullable = true)
 |-- thdib: double (nullable = true)
 |-- thdic: double (nullable = true)
 |-- desbV: double (nullable = true)
 |-- desbI: double (nullable = true)
 |-- kwhE: double (

In [158]:
# verify the persisted DataFrame
test_df.select('timestamp', 'kw', 'thdia').show(24)

+-------------------+------------------+-------------------+
|          timestamp|                kw|              thdia|
+-------------------+------------------+-------------------+
|2016-01-01 00:00:00|1324.2366666666667|            0.09315|
|2016-01-01 01:00:00|1245.2316666666666|0.09356666666666667|
|2016-01-01 02:00:00|1145.6866666666667|             0.0936|
|2016-01-01 03:00:00|1044.6466666666668|             0.0919|
|2016-01-01 04:00:00| 960.6921666666667|0.08845000000000001|
|2016-01-01 05:00:00| 911.6516666666666|0.08644999999999999|
|2016-01-01 06:00:00| 871.7088333333332|0.08438333333333332|
|2016-01-01 07:00:00| 775.2703333333334|0.09126666666666666|
|2016-01-01 08:00:00| 756.7946666666667|0.09646666666666666|
|2016-01-01 09:00:00| 821.0078333333332|0.09953333333333335|
|2016-01-01 10:00:00| 876.7158333333333|0.09881666666666668|
|2016-01-01 11:00:00| 948.4540000000001|0.09711666666666667|
|2016-01-01 12:00:00| 971.0126666666666|            0.09475|
|2016-01-01 13:00:00| 99

In [ ]:
# ToDo: trim this dataset to 2018-07-31 23:50:00 to adjust lenght to entire months

In [ ]:
# ToDo: save the dataset to 952 folders named after the date